# Implementing the `MiniBach` model - Part 3

In this step, we take the one-hot-encoded input and output vectors from Part 2 to train the neural network.

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import music21

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [3]:
x = np.load('input.npy')
y = np.load('output.npy')

input_length = x.shape[1]
output_length = y.shape[1]

In [4]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(input_length,)),
    keras.layers.Dense(400, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(output_length)
])

In [5]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])

In [6]:
model.fit(x, y, epochs=20)

Epoch 1/20
105/105 [==============================] - 2s 15ms/step - loss: 0.1480 - binary_accuracy: 0.9812
Epoch 2/20
105/105 [==============================] - 1s 13ms/step - loss: 0.1308 - binary_accuracy: 0.9841
Epoch 3/20
105/105 [==============================] - 1s 14ms/step - loss: 0.1273 - binary_accuracy: 0.9842
Epoch 4/20
105/105 [==============================] - 1s 13ms/step - loss: 0.1248 - binary_accuracy: 0.9842
Epoch 5/20
105/105 [==============================] - 1s 13ms/step - loss: 0.1232 - binary_accuracy: 0.9843
Epoch 6/20
105/105 [==============================] - 1s 13ms/step - loss: 0.1214 - binary_accuracy: 0.9844
Epoch 7/20
105/105 [==============================] - 1s 14ms/step - loss: 0.1213 - binary_accuracy: 0.9842
Epoch 8/20
105/105 [==============================] - 1s 14ms/step - loss: 0.1197 - binary_accuracy: 0.9842
Epoch 9/20
105/105 [==============================] - 1s 14ms/step - loss: 0.1180 - binary_accuracy: 0.9841
Epoch 10/20
105/105 [=======

In [8]:
model.save('trained_model.h5')

In [18]:
soprano = x[200]
predictions = model.predict(soprano.reshape(1, -1))

predictions = predictions.reshape(-1)
predictions = y[200]

soprano = soprano.reshape(64, -1)
alto = predictions[:1536].reshape(64, -1)
tenor = predictions[1536:3008].reshape(64, -1)
bass = predictions[3008:4928].reshape(64, -1)

In [10]:
display(alto.shape)
display(tenor.shape)
display(bass.shape)

(64, 24)

(64, 23)

(64, 30)

In [11]:
SOPRANO_MIN = 57
SOPRANO_MAX = 81

ALTO_MIN = 52
ALTO_MAX = 74

TENOR_MIN = 48
TENOR_MAX = 69

BASS_MIN = 36
BASS_MAX = 64

ranges = {
    'soprano': {midinumber: (midinumber - SOPRANO_MIN + 1) for midinumber in range(SOPRANO_MIN, SOPRANO_MAX + 1)},
    'alto': {midinumber: (midinumber - ALTO_MIN + 1) for midinumber in range(ALTO_MIN, ALTO_MAX + 1)},
    'tenor': {midinumber: (midinumber - TENOR_MIN + 1) for midinumber in range(TENOR_MIN, TENOR_MAX + 1)},
    'bass': {midinumber: (midinumber - BASS_MIN + 1) for midinumber in range(BASS_MIN, BASS_MAX + 1)},
}

reverse_ranges = {
    'soprano': {(midinumber - SOPRANO_MIN + 1): midinumber for midinumber in range(SOPRANO_MIN, SOPRANO_MAX + 1)},
    'alto': {(midinumber - ALTO_MIN + 1): midinumber for midinumber in range(ALTO_MIN, ALTO_MAX + 1)},
    'tenor': {(midinumber - TENOR_MIN + 1): midinumber for midinumber in range(TENOR_MIN, TENOR_MAX + 1)},
    'bass': {(midinumber - BASS_MIN + 1): midinumber for midinumber in range(BASS_MIN, BASS_MAX + 1)},
}

In [12]:
def decode_note(n, rang):
    if n == 0:
        ret = '--'
    else:
        note = music21.note.Note(type='16th')        
        note.pitch.midi = reverse_ranges[rang][n]        
        ret = note
    return ret

In [13]:
generation = {
    'soprano': [],
    'alto': [],
    'tenor': [],
    'bass': []
}

for sixteenth in range(64):
    soprano_note = decode_note(np.argmax(soprano[sixteenth]), 'soprano')
    if soprano_note == '--':
        last_note = generation['soprano'][-1]
        soprano_note = music21.note.Note(last_note.pitch.nameWithOctave, type='16th')
        if last_note.tie:
            soprano_note.tie = music21.tie.Tie('continue')
        else:
            last_note.tie = music21.tie.Tie('start')
            generation['soprano'][-1] = last_note
            soprano_note.tie = music21.tie.Tie('continue')
    else:
        if sixteenth > 0:
            last_note = generation['soprano'][-1]
            if last_note.tie:
                last_note.tie.type = 'end'
    generation['soprano'].append(soprano_note)
    
    alto_note = decode_note(np.argmax(alto[sixteenth]), 'alto')
    if alto_note == '--':
        last_note = generation['alto'][-1]
        alto_note = music21.note.Note(last_note.pitch.nameWithOctave, type='16th')
        if last_note.tie:
            alto_note.tie = music21.tie.Tie('continue')
        else:
            last_note.tie = music21.tie.Tie('start')
            generation['alto'][-1] = last_note
            alto_note.tie = music21.tie.Tie('continue')
    else:
        if sixteenth > 0:
            last_note = generation['alto'][-1]
            if last_note.tie:
                last_note.tie.type = 'end'
    generation['alto'].append(alto_note)
    
    tenor_note = decode_note(np.argmax(tenor[sixteenth]), 'tenor')
    if tenor_note == '--':
        last_note = generation['tenor'][-1]
        tenor_note = music21.note.Note(last_note.pitch.nameWithOctave, type='16th')
        if last_note.tie:
            tenor_note.tie = music21.tie.Tie('continue')
        else:
            last_note.tie = music21.tie.Tie('start')
            generation['tenor'][-1] = last_note
            tenor_note.tie = music21.tie.Tie('continue')
    else:
        if sixteenth > 0:
            last_note = generation['alto'][-1]
            if last_note.tie:
                last_note.tie.type = 'end'
    generation['tenor'].append(tenor_note)
    
    bass_note = decode_note(np.argmax(bass[sixteenth]), 'bass')
    if bass_note == '--':
        last_note = generation['bass'][-1]
        bass_note = music21.note.Note(last_note.pitch.nameWithOctave, type='16th')
        if last_note.tie:
            bass_note.tie = music21.tie.Tie('continue')
        else:
            last_note.tie = music21.tie.Tie('start')
            generation['bass'][-1] = last_note
            bass_note.tie = music21.tie.Tie('continue')
    else:
        if sixteenth > 0:
            last_note = generation['bass'][-1]
            if last_note.tie:
                last_note.tie.type = 'end'
    generation['bass'].append(bass_note)        

In [14]:
df = pd.DataFrame(generation)

In [15]:
df

,soprano,alto,tenor,bass
0,<music21.note.Note C>,<music21.note.Note F>,<music21.note.Note B->,<music21.note.Note F>
1,<music21.note.Note C>,<music21.note.Note F>,<music21.note.Note B->,<music21.note.Note F>
2,<music21.note.Note C>,<music21.note.Note E->,<music21.note.Note B->,<music21.note.Note F>
3,<music21.note.Note C>,<music21.note.Note D>,<music21.note.Note B->,<music21.note.Note F>
4,<music21.note.Note C>,<music21.note.Note E->,<music21.note.Note A>,<music21.note.Note F>
5,<music21.note.Note C>,<music21.note.Note E->,<music21.note.Note A>,<music21.note.Note F>
6,<music21.note.Note C>,<music21.note.Note E->,<music21.note.Note A>,<music21.note.Note F>
7,<music21.note.Note C>,<music21.note.Note E->,<music21.note.Note A>,<music21.note.Note F>
8,<music21.note.Note B->,<music21.note.Note D>,<music21.note.Note B->,<music21.note.Note B->
9,<music21.note.Note B->,<music21.note.Note D>,<music21.note.Note B->,<music21.note.Note B->


In [16]:
s = music21.stream.Stream()
s.append(df.soprano.to_list())
a = music21.stream.Stream()
a.append(df.alto.to_list())
t = music21.stream.Stream()
t.append(df.tenor.to_list())
b = music21.stream.Stream()
b.append(df.bass.to_list())
stream = music21.stream.Stream([s,a,t,b])

In [17]:
stream.write('musicxml', 'here.musicxml')

'/home/napulen/dev/MiniBach/here.musicxml'